# nav-helper
 Simple iphone app for navigation assistance.

This documentation is generated from docs/reng/reverse-engineering.ipynb using nbconvert.

```bash
python3.9 -m jupyter nbconvert --to markdown ./docs/reng/reverse-engineering.ipynb --output ../../README.md
```

## Purpose

This app started out of frustration in identifying remote locations in the wilderness. The Sig Sauer Kilo5K has bluetooth connectivity for sending information on angle and distance of ranged targets. The purpose of this app is to provide missing features that are helpful when navigating in the wilderness.

Personal goals include:
- Understanding bluetooth communication.
- Improve my reverse engineering ability for packet data.
- Understand mobile app development ecosystem.
- Documenting my learning process.
- Document an undocumented proprietary protocol.

Intended Features:
- Provide the GPS coordinates of a ranged point allowing a more accurate compass to enter the bearing of the ranged point.
- Optionally use the internal compass to set the bearing using the edge of the device.
- Convert between magnetic and true north using NCEI data.
- Improve accuracy of the ranged point by utilizing the angle and localized high-definition elevation data. (when available)
- Ability to log ranged points in the background, along with current GPS coordinates.
- GPS current position averaging to get a more accurate location.
- Ability to easily convert GPS coordinates between formats.
- Operate when offline.

## Progress

This section is notes on steps I am taking and information that I collect.

- [Capture bluetooth packets.](https://www.bluetooth.com/blog/a-new-way-to-debug-iosbluetooth-applications/)
    - Separate packet captures have been made, one for [pairing](docs/captures/kilo5k-pairing.json) and one for [ranging](docs/captures/kilo5k-range-11.2y-5deg.json). I expect further captures will be necessary to identify how the data is structured for the ranging information.
- Packets were saved from PacketLogger to be viewed in Wiresshark.

### Bluetooth Packets
RF = Range Finder
P = Phone

Ranging 11.2y 5 deg
1. RF->P MTU Request (Rx MTU: 244)
2. P->RF MTU Response (Rx MTU: 244)
3. P->RF L2CAP (???)
4. P->RF Read By Type GATT Characteristic Declaration
5. RF->P Rcvd Error Response Attribute not found (This is consistant for every range)
6. P->RF Write Request Handle: 0x0010 Value: 0100


In [ ]:
## Reverse engineering the bluetooth communication for the Kilo5K

In [38]:
import json
from rich import traceback
from rich import print

traceback.install()

packets = json.load(open('../captures/kilo5k-range-11.2y-5deg.json', 'r'))


def parse(input_hex):
    input_list = bytes.fromhex(input_hex).decode('ASCII').split(',')
    desc = {
            0: 'Unknown0',
            1: 'Unknown1',
            2: 'Unknown2',
            3: 'Unknown3',
            4: 'Unknown4',
            5: 'Range(y)5',
            6: 'Unknown6',
            7: 'Unknown7',
            8: 'Bearing?8',
            9: 'Unknown9',
            10: 'Unknown10',
            11: 'Angle(up or down?)',
            12: 'End?'
           }
    vals = dict()
    print(len(input_list))
    for i in range(len(input_list)):
        vals[desc[i]] = input_list[i]
    return vals


In [39]:
data = parse(packets[-2]['_source']['layers']['btatt']['btatt.value'].replace(':',''))
print(data)

13

{
    'Unknown0': ':AB',
    'Unknown1': '2',
    'Unknown2': '0.000',
    'Unknown3': '0.000',
    'Unknown4': '0.000',
    'Range(y)5': '11.200',
    'Unknown6': '0.000',
    'Unknown7': '0.000',
    'Bearing?8': '270.000',
    'Unknown9': '0.000',
    'Unknown10': '0.000',
    'Angle(up or down?)': '4.564',
    'End?': 'B5\r'
}

In [35]:
print(bytes.fromhex(packets[-2]['_source']['layers']['btatt']['btatt.value'].replace(':','')).decode('ASCII').split(','))

[
    ':AB',
    '2',
    '0.000',
    '0.000',
    '0.000',
    '11.200',
    '0.000',
    '0.000',
    '270.000',
    '0.000',
    '0.000',
    '4.564',
    'B5\r'
]